### This notebook uses a breathing actuation sequence that computes triggers inflation deflation based upon an average inhale/exhale duration and a scale factor

In [1]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import biofeatures
import threading
import numpy as np
import time

### Actuator config

In [5]:
actuator_ip = '192.168.0.110'
actuator_port = 12000

osc_client = SimpleUDPClient(actuator_ip, actuator_port) 
osc_client.send_message("/actuator/inflate", 100.0)
osc_client.send_message("/actuator/inflate", -100.0)
osc_client.send_message("/actuator/inflate", 0.0)
# osc_client.send_message("/actuator/inflate", 0.0)

### Data processing

In [ ]:
# def data_handler(unused_addr, args, data1, data2, data3, data4, data5, data6): #BITalino ServerBIT format
# def data_handler(unused_addr, args, values): #R-IoT Node-RED format
def data_handler(unused_addr, args, *values): #R-IoT 

    global ecg_data, resp_data
    global osc_client
    
    B = args[0]
    HR = args[1]
    
    # print(data5) # breathing
    # print(data6) # ECG
    
    # BITalino ServerBIT
#     ecg_data.append(float(data6))
#     resp_data.append(float(data5))

    # R-IoT Node-RED
#     resp_data.append(np.float(values.split("\t")[5]))
#     ecg_data.append(np.random.rand(1)[0])


#     ecg_data.append(float(values[13]))
    ecg_data.append(np.random.rand(1)[0])
    resp_data.append(float(values[12]))
#     print(resp_data[-1])
        
    if B.is_warmed_up:
        B.set_data(resp_data[-B.buffer_length:])
    
    if HR.is_warmed_up:
        HR.set_data(ecg_data[-HR.buffer_length:])
    


In [ ]:
def warmup(B, resp_data, HR, ecg_data):
    """Function to launch once the system has warmed up.
    Sets the  data initially and launches a recursive update of features.
    Parameters
    ----------
    B: object containing breathing data and features
    resp_data: respiration signal data to set
    HR: object containing ECG data and features
    ecg_data: ecg signal data to set
    """
    
    if not B.is_warmed_up:
        print("Breathing WARMUP")
        B.set_data(resp_data[-B.buffer_length:])
        B.is_warmed_up = True
        B.update_loop()
        
    if not HR.is_warmed_up:
        print("ECG WARMUP")
        HR.set_data(ecg_data[-HR.buffer_length:])
        HR.is_warmed_up = True
        HR.update_loop()
        
    breathing_factor = 1.2
    
    timer_actuation = threading.Timer(0.1, breathing_actuation, [B, breathing_factor, True] )
    timer_actuation.start()

In [ ]:
def breathing_actuation(B, breathing_factor, inflate):
    global osc_client
    global actuation_flag
    
    if not actuation_flag:
        return
    
    if inflate:
        print("inhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/inflate", 100.0)
        timer_exhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, breathing_actuation, [B, breathing_factor, False])
        timer_exhale.start()

    else:
        print("exhale: ", B.features['avg_exhale'])
        osc_client.send_message("/actuator/inflate", -100.0)
        timer_inhale = threading.Timer(B.features['avg_exhale'] * breathing_factor, breathing_actuation, [B, breathing_factor, True])
        timer_inhale.start()

In [ ]:
# riot_ip = '192.168.1.100'
# riot_ssid = 'riot'
def riot_data_handler(unused_addr, args, *values): #R-IoT 

    global ecg_data, resp_data
    global osc_client
    
    B = args[0]
    HR = args[1]
    
    ecg_data.append(float(values[13]))
    resp_data.append(float(values[12]))
    print(resp_data[-1])


def riot_listener(ip, port):

    riot_dispatcher = dispatcher.Dispatcher()
    riot_dispatcher.map("/*/raw", assign_riot_data)
    # riot_dispatcher.map("/*/bitalino", assign_bitalino_data)

    # server = osc_server.ThreadingOSCUDPServer(
    #   (ip, port), riot_dispatcher)
    server = osc_server.ThreadingOSCUDPServer(
      (ip, port), riot_dispatcher)
    print("Serving on {}".format(server.server_address))
    ut.osc_server_started = True
    server.serve_forever()
    
def assign_riot_data(unused_addr, *values):
    d_id = (int(unused_addr[1]))
    if d_id not in ut.device_ids: new_device(d_id)

    channels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
    labels = ["ACC_X", "ACC_Y", "ACC_Z", "GYRO_X", "GYRO_Y", "GYRO_Z", "MAG_X", "MAG_Y", "MAG_Z",
        "TEMP", "IO", "A1", "A2", "C", "Q1", "Q2", "Q3", "Q4", "PITCH", "YAW", "ROLL", "HEAD"]
    ch_mask = numpy.array(channels) - 1
    try:
        cols = numpy.arange(len(ch_mask))
        res = "{"
        for i in cols:
            res += '"' + labels[i] + '":' + str(values[i]) + ','
        res = res[:-1] + "}"
        #if len(cl) > 0: cl[-1].write_message(res)
        ut.device_data[d_id] = res
    except:
        traceback.print_exc()
        os._exit(0)
    

In [ ]:
riot_ip = "192.168.0.102"
riot_port = 31000 


In [ ]:
ecg_data = []
resp_data = []
times_run = 0
B = biofeatures.breathing(data = np.ones(10), buffer_length=2000, srate=100)
HRV = biofeatures.hrv(data = np.ones(10), buffer_length=2000, srate=100)
my_dispatcher = dispatcher.Dispatcher()
my_dispatcher.map("/*/raw", data_handler, B, HRV)

In [ ]:
#Enable actuation
actuation_flag = True

# How many times we've run
times_run += 1

# Warmup
warmup_t = 10
timer_warmup = threading.Timer(warmup_t, warmup, [B,resp_data,HRV,ecg_data])
timer_warmup.start()

# Start the server
server = osc_server.ThreadingOSCUDPServer((riot_ip, riot_port), my_dispatcher)
print(time.strftime("%H:%M:%S", time.gmtime()))
print("Serving on {}".format(server.server_address))

try: 
    server.serve_forever()
except KeyboardInterrupt:
    osc_client.send_message("/actuator/inflate", 0.0)
    B.update_data_flag = False
    HRV.update_data_flag = False
    actuation_flag = False
    server.server_close()
    
    # Safeguard in order to run more than once
    B.is_warmed_up = False
    HRV.is_warmed_up = False

    # TODO 
    # we lose access to timer_actuation --> we're never able to STOP it. 
    # The second time, it is still running. We'd need an extra flag to control it
except:
    raise